# Whole Genome Trio Analysis Example

An example of using pyvariantfilter to find candidate variants in a WGS trio.

In [1]:
from pyvariantfilter.family import Family
from pyvariantfilter.family_member import FamilyMember
from pyvariantfilter.variant_set import VariantSet
from pyvariantfilter.wrappers import run_phen2gene

## Create A Family

The first step in an analyis is to create a Family object which describes the relationships between your samples. In this case the proband is female and affected whilst both parents are unaffected.

In [2]:
# Create family members - arguments are (id, family_id, sex(1=male, 2=female) and affected status)
mum = FamilyMember('mum', 'FAM001', 2, False)
dad = FamilyMember('dad', 'FAM001', 1, False)
proband = FamilyMember('proband', 'FAM001', 2, True, mum=mum, dad=dad)

my_family = Family('FAM001')

my_family.add_family_member(mum)
my_family.add_family_member(dad)
my_family.add_family_member(proband)

# Set the proband - we need to do this in order to know which sample to look in for de novos etc
my_family.set_proband(proband.get_id())

## Load Variants From VCF

In [3]:
# Create a new VariantSet object
my_variant_set = VariantSet()

# Associate the my_family object with my_variant_set
my_variant_set.add_family(my_family)

### Create an Initial Filtering Function

For whole genome analysis we may not be able to load all the variants into memory so we define an initial filtering function so that only certain variants are loaded into the variant set. This function must take a Variant object as the first argument and must return either True or False.

In [4]:
def passes_initial_filter(variant, proband_id):
    """
    Filter variants from the VCF.
    
    We import if the variant passes quality filtering and is below 1% in gnomad exomes and gnomad genomes AND
    
    a) Has a predicted affect on splicing of > 80% OR
    b) Is listed as pathogenic in clinvar OR
    c) Has a a relevant consequence
    
    """
    # If the proband has the variant and we pass the genotype and variant level filters
    if variant.has_alt(proband_id) and variant.passes_gt_filter(proband_id, min_gq=20) and variant.passes_filter():
        
        
        # The filter_on_numerical_transcript_annotation_lte() function allows us to filter on numerical values 
        # we can set different cutoffs for different variant types. For example ad_het is variants in which the 
        # proband is heterozygous on an autosome. In this case we get two boolean values describing whether the 
        # variant is below 1% in the gnomad genomes and gnomad exomes datasets.
        freq_filterg = variant.filter_on_numerical_transcript_annotation_lte(annotation_key='gnomADg_AF_POPMAX',
                                                                                          ad_het=0.01,
                                                                                          ad_hom_alt=0.01,
                                                                                          x_male =0.01,
                                                                                          x_female_het=0.01,
                                                                                          x_female_hom=0.01,
                                                                                          compound_het=0.01,
                                                                                          y=0.01,
                                                                                          mt=0.01,
                                                                                          )
        freq_filtere = variant.filter_on_numerical_transcript_annotation_lte(annotation_key='gnomADe_AF_POPMAX',
                                                                                          ad_het=0.01,
                                                                                          ad_hom_alt=0.01,
                                                                                          x_male =0.01,
                                                                                          x_female_het=0.01,
                                                                                          x_female_hom=0.01,
                                                                                          compound_het=0.01,
                                                                                          y=0.01,
                                                                                          mt=0.01,
                                                                                          )  
        
        
        # Get some annotations from SpliceAI - use agg_func = max in case different transcripts have 
        # different values.
        SpliceAI_DS_AG = variant.get_numerical_transcript_annotation('SpliceAI_DS_AG', agg_func='max')
        SpliceAI_DS_AL = variant.get_numerical_transcript_annotation('SpliceAI_DS_AL', agg_func='max')
        SpliceAI_DS_DG = variant.get_numerical_transcript_annotation('SpliceAI_DS_DG', agg_func='max')
        SpliceAI_DS_DL =  variant.get_numerical_transcript_annotation('SpliceAI_DS_DL', agg_func='max')
        
        max_splice = max(SpliceAI_DS_AG, SpliceAI_DS_AL, SpliceAI_DS_DG, SpliceAI_DS_DL )
        
        # If the variant is below 1% and the we have more than 80% chance of affect on splicing then import
        if freq_filterg and freq_filtere and max_splice >0.8:
                        
            return True
        
        # Coopt the get_genes() function to get the clinvar annotation VEP field.
        clinvar = variant.get_genes(feature_key='CLIN_SIG')
        is_path_in_clinvar = False
        
        for anno in clinvar:
            
            if 'pathogenic' in anno.lower():
                is_path_in_clinvar = True
                break
                
        # If the variant is below 1% and pathogenic in clinvar then import
        if freq_filterg and freq_filtere and is_path_in_clinvar:
            
            return True
        
        csq_filter = False
        
        if variant.get_worst_consequence() in {'transcript_ablation': None,
                                               'splice_acceptor_variant': None,
                                               'splice_donor_variant': None,
                                               'stop_gained': None,
                                               'frameshift_variant': None,
                                               'stop_lost': None,
                                               'start_lost': None,
                                               'transcript_amplification': None,
                                               'inframe_insertion': None,
                                               'inframe_deletion': None,
                                               'missense_variant': None,
                                               'protein_altering_variant': None,
                                               'incomplete_terminal_codon_variant': None,
                                               'start_retained_variant': None,
                                               'stop_retained_variant': None}:
        
            csq_filter = True
        
       # If the variant is below 1% and has a relevant consequence then import
        if csq_filter and freq_filterg and freq_filtere:
            
            return True
        
    return False

In [5]:
# Import variants from a Platypus VCF - and apply our filtering function. Note that arguments to the 
# filtering function can be passed using the args argument which should be a tuple. The first variant argument to 
# passes_initial_filter does not need to be added to the args argument.

my_variant_set.read_variants_from_platypus_vcf('test_data/123000015_merged_vep.fixed.vcf.gz',
                                               proband_variants_only=True,
                                               filter_func=passes_initial_filter,
                                               args=(proband.get_id(),))

In [6]:
print (f'{len(my_variant_set.variant_dict)} variants have been loaded into the variant set.')

629 variants have been loaded into the variant set.


## Get Compound Hets

Now we have a VariantSet object loaded with variants we can find compound hets. There are different methods for this depending on whether the proband has both parents or not. Since in this case we do - we can phase the compound hets so we only report compound het pairs where each variant within the pair has been inherited from a different parent.

In [7]:
# Create an attribute my_variant_set.candidate_compound_het_dict where each transcript is a key the variants 
# within that transcript are the values
my_variant_set.get_candidate_compound_hets()

# As we have both parents we can filter the compound hets.
my_variant_set.filter_compound_hets()

# Flatten the filtered compound hets (my_variant_set.filtered_compound_het_dict) into a a dictionary with each 
# genuine compound het as a key.
my_variant_set.get_filtered_compound_hets_as_dict()

In [8]:
print (f'The first variant in the final compound het dict is: {list(my_variant_set.final_compound_hets.keys())[0]}')

The first variant in the final compound het dict is: 2:151533488C>T


## Apply Inheritance Filter

We want to find variants which match the de novo, autosomal reccessive, compound_het and X linked reccessive models. Each variant in the variant set has methods available to find these as shown below.

In [9]:
def passes_final_filter(variant, compound_het_dict):
    
    # Get variants which match certain inheritance models
    if variant.matches_inheritance_model(['autosomal_dominant',
                                          'autosomal_reccessive',
                                          'x_reccessive',
                                          'x_dominant',
                                          'de_novo',
                                          'compound_het'], compound_het_dict):
        
        return True
        
    return False

In [10]:
# Apply a the passes_final_filter() function

my_variant_set.filter_variants(passes_final_filter, args=(my_variant_set.final_compound_hets,))

In [11]:
# Convert to dataframe - VEP fields get 'csq_' as a prefix. Each transcript that a variant is in gets its own row.

df = my_variant_set.to_df()

In [13]:
print (f'{len(my_variant_set.variant_dict)} variants left after filtering.')

27 variants left after filtering.


## Prioritise Genes

The package contains wrappers for the Phenolyzer and Phen2Gene programs. These will have to be installed separately.

These allow the Prioritisation of genes within the candidates we have found.

In [14]:
phen2gene_dir = '/home/joseph/Documents/apps/Phen2Gene/'
temp_dir= '/home/joseph/Documents/pyvariantfilter/temp'
job_name = 'FAM001'
hpo_terms = ['HP:0002342', 'HP:0002194',
             'HP:0000750', 'HP:0004422',
             'HP:0000414', 'HP:0000490',
             'HP:0007930', 'HP:0010862',
             'HP:0011343']

In [15]:
gene_scores = run_phen2gene(phen2gene_dir=phen2gene_dir,
                            temp_dir=temp_dir,
                            job_name=job_name,
                            hpo_terms=hpo_terms)

In [16]:
def apply_score_to_df(df, gene_scores):
    
    gene = df['csq_SYMBOL']
    score = 0
    if gene in gene_scores:
        
        score = gene_scores[gene]
    
    return float(score)

In [17]:
df['gene_score'] = df.apply(apply_score_to_df, axis=1, args=(gene_scores,))

In [20]:
# View top variant
df[['variant_id',
    'csq_SYMBOL',
    'worst_consequence',
    'inheritance_models',
    'csq_PICK',
    'gene_score']][(df['csq_PICK']== '1')].sort_values('gene_score', ascending=False).head(1)

,variant_id,csq_SYMBOL,worst_consequence,inheritance_models,csq_PICK,gene_score
1,1:153816379AG>A,GATAD2B,frameshift_variant,autosomal_dominant|de_novo,1,0.220934
